In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import plotly.express as px
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import os

import time
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras import backend as K

from PIL import Image
import pandas as pd
import csv
print(tf.__version__)
from tensorflow.python.client import device_lib

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
# print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
Num GPUs Available:  1


In [2]:
# image processing methods
def process_image(imgtensor):
    imgtensor *= (1/255) #
    return imgtensor

def deprocess_image(imgtensor):
    imgtensor *= 255
    imgtensor = np.clip(imgtensor, 0, 255)
    return imgtensor

In [3]:
VAE = keras.models.load_model(r"C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\VAE_punctured_const_var_Z3D")

In [4]:
#punctured dataset:
train_dir = 'D:/Datasets/VAE_zeroshot/data_punctured_1p25-1_R0p3_RESIZE/train' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
validation_dir = 'D:/Datasets/VAE_zeroshot/data_punctured_1p25-1_R0p3_RESIZE/test' #

batch_size = 8


#create images dataset
#set shuffle to False since this dataset must match up with the weights dataset (which isnt shuffled)
img_dataset_train = image_dataset_from_directory(
    train_dir, shuffle = False, image_size=(256, 256), batch_size=8, color_mode='grayscale', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset_train = img_dataset_train.map(process_image)
print(img_dataset_train.cardinality()) #set this value as the steps_per_epoch in .fit


#set up validation dataset

#create images dataset
img_dataset_validation = image_dataset_from_directory(
    validation_dir, shuffle = False, image_size=(256, 256), batch_size=8, color_mode='grayscale', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset_validation = img_dataset_validation.map(process_image)
print(img_dataset_validation.cardinality()) #set this value as the number of validation steps in .fit


Found 4299 files belonging to 1 classes.
tf.Tensor(538, shape=(), dtype=int64)
Found 211 files belonging to 1 classes.
tf.Tensor(27, shape=(), dtype=int64)
Found 15 files belonging to 1 classes.


In [20]:
#PREDICT images 

#visualize image reconstructions
validation_sample = img_dataset_validation.take(24)
# hole_sample = dataset_with_weights_hole.take(24)

print(img_dataset_transect.cardinality())
xhat = VAE.predict(img_dataset_transect, steps=15)
xhat = deprocess_image(xhat)
xhat = xhat.astype(np.uint8)
xhat.shape
for j in range(15):
    pil_im = Image.fromarray(xhat[j,:,:,0],mode='L') # convert each slice of predicted batch to a PIL image object  
    pil_im.save(r'C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\Transects\Parameter_space_transects\Xhat3\transect_point_' + f'{j+1:02}' + '.png')


Found 15 files belonging to 1 classes.
tf.Tensor(15, shape=(), dtype=int64)


In [8]:
latent_dim = 3
# make decoder model by copying VAE layers to a new model
VAE.layers[16]
decoder_input = Input(shape = (latent_dim), name='latent_vector')
x = decoder_input
for layer in VAE.layers[12:-1]:
    print(layer.name)
    x = layer(x)
    
decoder = Model(decoder_input, x, name='decoder')
decoder.summary()

dense
reshape
decoderupsample1
conv2d_transpose
batch_normalization
up_sampling2d
conv2d_transpose_1
batch_normalization_1
up_sampling2d_1
conv2d_transpose_2
batch_normalization_2
conv2d_transpose_3
batch_normalization_3
conv2d_transpose_4
Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
latent_vector (InputLayer)   [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 16384)             49152     
_________________________________________________________________
reshape (Reshape)            (None, 32, 32, 16)        0         
_________________________________________________________________
decoderupsample1 (UpSampling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 64, 64, 64)        9280      
_________________

In [5]:
#make ENCODER that skips the sampling layer and just returns z_mean

VAE.layers[14]
detached_encoder_input = Input(shape=(256, 256, 1), name='detached_encoder_input')
x = detached_encoder_input #
for layer in VAE.layers[1:11]:#[1:15]we must skip the VAE input layer (for loop starts at layer 1 instead of 0) as it causes problems to use the same input layer in two separate models
    print(layer.name)
    x = layer(x)

encoder = Model(detached_encoder_input, x, name='encoder')
encoder.summary()
# encoder.compile()

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
conv2d_2
max_pooling2d_2
conv2d_3
max_pooling2d_3
flatten
z_mean
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
detached_encoder_input (Inpu [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 128)       73856     
______________

In [10]:
#create Model for inference which skips the sample_latent_vector layer and just uses z_mean as the latent vector
VAE_2_input = Input(shape=(256, 256, 1), name='VAE_2_input')
VAE_2 = Model(VAE_2_input, decoder(encoder(VAE_2_input)), name="VAE_sampless")
VAE_2.summary()


Model: "VAE_sampless"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
VAE_2_input (InputLayer)     [(None, 256, 256, 1)]     0         
_________________________________________________________________
encoder (Functional)         (None, 3)                 345411    
_________________________________________________________________
decoder (Functional)         (None, 256, 256, 1)       204545    
Total params: 549,956
Trainable params: 548,932
Non-trainable params: 1,024
_________________________________________________________________


NameError: name 'VAE' is not defined

In [85]:
data_sample = img_dataset_train.take(300)
Z_data_sample = encoder.predict(data_sample)
Z_data_sample.shape

(15, 3)

In [90]:
# set up transect images dataset
transect_dir = r'C:\Users\MrLin\Documents\Experiments\VAE_zeroshot\Transects\Parameter_space_transects\X3_resize' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
#set shuffle to False since this dataset must match up with the weights dataset (which isnt shuffled)
img_dataset_transect = image_dataset_from_directory(
    transect_dir, shuffle = False, image_size=(256, 256), batch_size=1, color_mode='grayscale', validation_split=None, label_mode=None)
#rescale pixel values
img_dataset_transect = img_dataset_transect.map(process_image)

Z3_transect = encoder.predict(img_dataset_transect)

Z1_transect.shape

Found 15 files belonging to 1 classes.


(15, 3)

In [238]:
import plotly.graph_objects as go
# data = {'xx':Z_data_sample[:,0],
#        'yy':Z_data_sample[:,1],
#        'zz':Z_data_sample[:,2],
#        'size':.1*np.ones(shape=Z_data_sample[:,1].shape),
#        'color':['image' for j in range(len(Z_data_sample))],
#        'symbol':['img' for j in range(len(Z_data_sample))]}

# Z_transect_df = pd.DataFrame(data)
# symbol_map = {'transect':'square',
#              'img':'circle'}
# color_map = {'image': 'black', 'transect':'red'}


# fig = px.scatter_3d(Z_transect_df, name='images', x='x', y='y', z='z', size='size', color='color', symbol ='symbol', symbol_map=symbol_map,
#                     size_max=5, opacity=.8, color_discrete_map=color_map)


fig2 = go.Figure(data=[go.Scatter3d(
    name='training images',
    x=Z_data_sample[:,0],
    y=Z_data_sample[:,1],
    z=Z_data_sample[:,2], 
    mode='markers',
    marker=dict(
        size=1.5, color='black',symbol='circle')
)])


Z_transect_df


fig2.add_trace(go.Scatter3d(name='P1',
    x=Z1_transect[:,0], y=Z1_transect[:,1], z=Z1_transect[:,2],
    marker=dict(
        size=4,
        color='blue',
    ),
    line=dict(
        color='blue',
        width=4
    )
))

fig2.add_trace(go.Scatter3d(name='P2',
    x=Z2_transect[:,0], y=Z2_transect[:,1], z=Z2_transect[:,2],
    marker=dict(
        size=4,
        color='green',
    ),
    line=dict(
        color='green',
        width=4
    )
))

fig2.add_trace(go.Scatter3d(name='P3',
    x=Z3_transect[:,0], y=Z3_transect[:,1], z=Z3_transect[:,2],
    marker=dict(
        size=4,
        color='red',
    ),
    line=dict(
        color='red',
        width=4
    )
))


fig2.update_layout(
    width=900,
    height=700,
    scene_xaxis_title_text='\u03BC<sub>1</sub>',
    scene_yaxis_title_text="\u03BC<sub>2</sub>",
    scene_zaxis_title_text="\u03BC<sub>3</sub>",
                
#     font=dict( #setting global font will override the xaxis properties set
#         family="Courier New, monospace",
#         color="RebeccaPurple"
#     ),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.17,
        font_size=16
    ),
    scene_xaxis = dict(tickfont=dict(size=12),
                title_font_size=20),
    scene_yaxis = dict(tickfont=dict(size=12),
                title_font_size=20),
    scene_zaxis = dict(tickfont=dict(size=12),
                title_font_size=20),
)

fig2.show()

In [230]:
fig2.layout


Layout({
    'height': 700,
    'legend': {'x': 0.17, 'xanchor': 'left', 'y': 0.99, 'yanchor': 'top'},
    'scene': {'xaxis': {'title': {'text': 'μ<sub>1</sub>'}},
              'yaxis': {'title': {'text': 'μ<sub>2</sub>'}},
              'zaxis': {'title': {'text': 'μ<sub>3</sub>'}}},
    'template': '...',
    'width': 900,
    'xaxis': {'tickfont': {'size': 10}, 'title': {'font': {'size': 20}}}
})